In [14]:
import pandas as pd

articles = pd.read_csv('../get_articles/all_articles.csv')

articles.head(1)

,authors,title,date,year,edition,url,text
0,NaN,本期导读,2024-01-01,2024.0,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...


In [15]:

def determine_edition_quarter(edition_number):
    if edition_number in range(1,7):
        return 1
    elif edition_number in range(7,13):
        return 2
    elif edition_number in range(13, 19):
        return 3
    else:
        return 4
    
def combine_year_quarter(row):
    return str(row["year"]) + " Q" + str(row["quarter"])

# drop articles without a year
articles = articles.dropna(subset=["year"])
print(f"Number of articles after dropping missing years: {len(articles)}")
    
# add a column for the quarter
articles["quarter"] = articles['edition'].map(lambda x: determine_edition_quarter(x))

# add a column for the year/quarter combo
articles = articles.astype({"year": int})
articles["year_quarter"] = articles.apply(combine_year_quarter, axis=1)

Number of articles after dropping missing years: 2179


In [16]:
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter
0,NaN,本期导读,2024-01-01,2024,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...,1,2024 Q1


In [17]:
articles.to_csv("minimally_processed_articles.csv")

In [3]:
# preprocessing
import re
import jieba 

def remove_punctuation(text):
    try:
        return re.sub('[，。：；‘“’”！？（） 【】「」/、｜《》]', "", text)
    except:
        return ""

# remove punctuation
articles['cleaned_text'] = articles['text'].map(remove_punctuation)
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text
0,NaN,本期导读,2024-01-01,2024,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...,1,2024 Q1,本期发表了习近平总书记的重要文章以美丽中国建设全面推进人与自然和谐共生的现代化文章强调今后5...


In [4]:
# parse the Chinese words to have spaces
def parse_chinese(text):
    seg_text = jieba.cut(text)
    seg_list = [word for word in seg_text]
    return " ".join(seg_list)

articles['cleaned_text_wspaces'] = articles["cleaned_text"].map(lambda x: parse_chinese(x))

articles.head(1)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/r4/qqn8tv4d7g18t_5fsxt6y1340000gn/T/jieba.cache
Loading model cost 0.312 seconds.
Prefix dict has been built successfully.


,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces
0,NaN,本期导读,2024-01-01,2024,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...,1,2024 Q1,本期发表了习近平总书记的重要文章以美丽中国建设全面推进人与自然和谐共生的现代化文章强调今后5...,本期 发表 了 习近平 总书记 的 重要 文章 以 美丽 中国 建设 全面 推进 人与自然 ...


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/calebharding/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def get_ngrams(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ''.join(grams) for grams in n_grams]

articles['unigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 1))

In [7]:
articles['bigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 2))

In [8]:
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram
0,NaN,本期导读,2024-01-01,2024,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...,1,2024 Q1,本期发表了习近平总书记的重要文章以美丽中国建设全面推进人与自然和谐共生的现代化文章强调今后5...,本期 发表 了 习近平 总书记 的 重要 文章 以 美丽 中国 建设 全面 推进 人与自然 ...,"[本期, 发表, 了, 习近平, 总书记, 的, 重要, 文章, 以, 美丽, 中国, 建设...","[本期发表, 发表了, 了习近平, 习近平总书记, 总书记的, 的重要, 重要文章, 文章以..."


In [9]:
# create unigram count dictionary
def ngram_counts(ngram):
    # create word count dictionary
    word_count = {}

    for word in ngram:
        if word in word_count.keys():
            cur_val = word_count[word]
            cur_val += 1
            word_count[word] = cur_val
        else:
            word_count[word] = 1

    # sort the dictionary
    word_count = {k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}

    return word_count

articles['unigram_count'] = articles["unigram"].map(lambda x: ngram_counts(x))
articles['bigram_count'] = articles["bigram"].map(lambda x: ngram_counts(x))

articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram,unigram_count,bigram_count
0,NaN,本期导读,2024-01-01,2024,1.0,http://www.qstheory.cn/dukan/qs/2024-01/01/c_1...,本期发表了习近平总书记的重要文章《以美丽中国建设全面推进人与自然和谐共生的现代化》。文章强调...,1,2024 Q1,本期发表了习近平总书记的重要文章以美丽中国建设全面推进人与自然和谐共生的现代化文章强调今后5...,本期 发表 了 习近平 总书记 的 重要 文章 以 美丽 中国 建设 全面 推进 人与自然 ...,"[本期, 发表, 了, 习近平, 总书记, 的, 重要, 文章, 以, 美丽, 中国, 建设...","[本期发表, 发表了, 了习近平, 习近平总书记, 总书记的, 的重要, 重要文章, 文章以...","{'的': 24, '文章': 14, '建设': 11, '中国': 10, '美丽': ...","{'美丽中国': 9, '的文章': 9, '中国建设': 7, '习近平总书记': 4, ..."


In [10]:
articles.to_csv('processed_articles.csv', index=False)